In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['BP']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('blood pressure', 7623),
 ('benzo a pyrene', 328),
 ('bullous pemphigoid', 228),
 ('bisphosphonate', 124),
 ('bipolar disorder', 82),
 ('black phosphorus', 69),
 ('binding potential', 56),
 ('biological process', 49),
 ('basic protein', 38),
 ('brevipedicellus', 33),
 ('binding protein', 28),
 ('benzophenone', 25),
 ('bipolar', 24),
 ('bacterial production', 20),
 ('butylparaben', 17),
 ('bovine pericardium', 16),
 ('branch point', 14),
 ('benzpyrene', 14),
 ('butyl paraben', 13),
 ('benzoyl peroxide', 13),
 ('bee pollen', 12),
 ('biphenyl', 12),
 ('breakpoint', 12),
 ('brachial plexus', 11),
 ('blast phase', 11),
 ('benzopyrene', 11),
 ('beet pulp', 10),
 ('bench press', 9),
 ('breaking point', 9),
 ('benzo alpha pyrene', 9),
 ('biodegradable polymer', 8),
 ('back pain', 8),
 ('bodily pain', 8),
 ('ball peptide', 8),
 ('bisphenol', 7),
 ('basilar papilla', 6),
 ('blocking peptide', 6),
 ('branchpoint', 6),
 ('bell s palsy', 6),
 ('benzo α pyrene', 6),
 ('back propagation', 5),
 ('ba

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D001794': 'Blood Pressure',
 'MESH:D010391': 'Pemphigoid, Bullous',
 'CHEBI:CHEBI:77383': '1,1-bis(phosphonic acid)',
 'MESH:D001714': 'Bipolar Disorder',
 'CHEBI:CHEBI:28659': 'phosphorus atom',
 'GO:GO:0008150': 'biological_process',
 'CHEBI:CHEBI:41308': 'benzophenone',
 'CHEBI:CHEBI:88542': 'Butylparaben',
 'MESH:D010496': 'Pericardium',
 'CHEBI:CHEBI:29865': 'benzo[a]pyrene',
 'CHEBI:CHEBI:85122': 'paraben',
 'CHEBI:CHEBI:82405': 'Benzoyl peroxide',
 'MESH:D011058': 'Pollen',
 'CHEBI:CHEBI:17097': 'biphenyl',
 'MESH:D001917': 'Brachial Plexus',
 'MESH:D001752': 'Blast Crisis',
 'CHEBI:CHEBI:60027': 'polymer',
 'MESH:D001416': 'Back Pain',
 'MESH:D010146': 'Pain',
 'CHEBI:CHEBI:22901': 'bisphenol',
 'MESH:D020330': 'Bell Palsy',
 'CHEBI:CHEBI:33694': 'biomacromolecule',
 'CHEBI:CHEBI:18208': 'benzylpenicillin',
 'MESH:D010361': 'Patients',
 'MESH:D011429': 'Propolis',
 'CHEBI:CHEBI:35545': 'bipyridine',
 'CHEBI:CHEBI:3219': 'bupropion',
 'CHEBI:CHEBI:3044': 'benzphetamine',


In [22]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [23]:
result = [grounding_map, names, pos_labels]

In [24]:
result

[{'b phymatum stm815': 'ungrounded',
  'back pain': 'MESH:D001416',
  'back propagation': 'ungrounded',
  'backpropagation': 'ungrounded',
  'bacterial pneumonia': 'MESH:D018410',
  'bacterial production': 'ungrounded',
  'bacterioplankton production': 'ungrounded',
  'ball peptide': 'ungrounded',
  'banana peels': 'ungrounded',
  'band pass': 'ungrounded',
  'barometric pressure': 'ungrounded',
  'basal progenitor': 'ungrounded',
  'basic protein': 'ungrounded',
  'basilar papilla': 'ungrounded',
  'beat period': 'ungrounded',
  'bee pollen': 'ungrounded',
  'beet pulp': 'ungrounded',
  'bell s palsy': 'MESH:D020330',
  'bench press': 'ungrounded',
  'benzo a pyrene': 'CHEBI:CHEBI:29865',
  'benzo alpha pyrene': 'CHEBI:CHEBI:29865',
  'benzo α pyrene': 'CHEBI:CHEBI:29865',
  'benzophenone': 'ungrounded',
  'benzopyrene': 'ungrounded',
  'benzoyl peroxide': 'ungrounded',
  'benzphetamine': 'CHEBI:CHEBI:3044',
  'benzpyrene': 'CHEBI:CHEBI:29865',
  'benzylpenicillin': 'CHEBI:CHEBI:18208

In [25]:
grounding_map, names, pos_labels = [{'b phymatum stm815': 'ungrounded',
  'back pain': 'MESH:D001416',
  'back propagation': 'ungrounded',
  'backpropagation': 'ungrounded',
  'bacterial pneumonia': 'MESH:D018410',
  'bacterial production': 'ungrounded',
  'bacterioplankton production': 'ungrounded',
  'ball peptide': 'ungrounded',
  'banana peels': 'ungrounded',
  'band pass': 'ungrounded',
  'barometric pressure': 'ungrounded',
  'basal progenitor': 'ungrounded',
  'basic protein': 'ungrounded',
  'basilar papilla': 'ungrounded',
  'beat period': 'ungrounded',
  'bee pollen': 'ungrounded',
  'beet pulp': 'ungrounded',
  'bell s palsy': 'MESH:D020330',
  'bench press': 'ungrounded',
  'benzo a pyrene': 'CHEBI:CHEBI:29865',
  'benzo alpha pyrene': 'CHEBI:CHEBI:29865',
  'benzo α pyrene': 'CHEBI:CHEBI:29865',
  'benzophenone': 'ungrounded',
  'benzopyrene': 'ungrounded',
  'benzoyl peroxide': 'ungrounded',
  'benzphetamine': 'CHEBI:CHEBI:3044',
  'benzpyrene': 'CHEBI:CHEBI:29865',
  'benzylpenicillin': 'CHEBI:CHEBI:18208',
  'bereitschaftspotential': 'ungrounded',
  'biliopancreatic': 'ungrounded',
  'binding potential': 'ungrounded',
  'binding protein': 'ungrounded',
  'biodegradable polymer': 'ungrounded',
  'biological process': 'ungrounded',
  'biopharmaceutical': 'ungrounded',
  'biopolymer': 'CHEBI:CHEBI:33694',
  'biopterin': 'CHEBI:CHEBI:15373',
  'biparental': 'ungrounded',
  'biphenyl': 'CHEBI:CHEBI:17097',
  'bipolar': 'MESH:D001714',
  'bipolar disorder': 'MESH:D001714',
  'bipolar patients': 'MESH:D001714',
  'bipyridyl': 'CHEBI:CHEBI:35545',
  'birch pollen': 'ungrounded',
  'bisection point': 'ungrounded',
  'bisphenol': 'CHEBI:CHEBI:22901',
  'bisphosphonate': 'MESH:D004164',
  'black phosphorus': 'ungrounded',
  'blast phase': 'MESH:D001752',
  'blastic phase': 'ungrounded',
  'blocking peptide': 'ungrounded',
  'blood perfusion': 'MESH:D010477',
  'blood plasma': 'MESH:D010949',
  'blood pressure': 'MESH:D001794',
  'blueberry juice and probiotics': 'MESH:D019936',
  'bodily pain': 'MESH:D010146',
  'bodipy ® 665 676': 'ungrounded',
  'body project': 'ungrounded',
  'bovine pericardium': 'MESH:D010496',
  'bowenoid papulosis': 'ungrounded',
  'boysenberry polyphenol': 'CHEBI:CHEBI:26195',
  'brachial plexus': 'ungrounded',
  'branch point': 'ungrounded',
  'branchpoint': 'ungrounded',
  'brazilian propolis': 'MESH:D011429',
  'breaking point': 'ungrounded',
  'breakpoint': 'ungrounded',
  'brevipedicellus': 'UP:P46639',
  'bromopyruvate': 'MESH:C017092',
  'bronchopneumonia': 'MESH:D001996',
  'bruce peninsula': 'ungrounded',
  'buckypaper': 'ungrounded',
  'bullous pemphigoid': 'MESH:D010391',
  'bupropion': 'CHEBI:CHEBI:3219',
  'burkholderia pseudomallei': 'MESH:D016957',
  'butyl paraben': 'CHEBI:CHEBI:85122',
  'butylparaben': 'CHEBI:CHEBI:88542',
  'butylphenol': 'ungrounded'},
 {'MESH:D001416': 'Back Pain',
  'MESH:D018410': 'Pneumonia, Bacterial',
  'MESH:D020330': 'Bell Palsy',
  'CHEBI:CHEBI:29865': 'benzo[a]pyrene',
  'CHEBI:CHEBI:3044': 'benzphetamine',
  'CHEBI:CHEBI:18208': 'benzylpenicillin',
  'CHEBI:CHEBI:33694': 'biomacromolecule',
  'CHEBI:CHEBI:15373': 'biopterin',
  'CHEBI:CHEBI:17097': 'biphenyl',
  'MESH:D001714': 'Bipolar Disorder',
  'CHEBI:CHEBI:35545': 'bipyridine',
  'CHEBI:CHEBI:22901': 'bisphenol',
  'MESH:D004164': 'Diphosphonates',
  'MESH:D001752': 'Blast Crisis',
  'MESH:D010477': 'Perfusion',
  'MESH:D010949': 'Plasma',
  'MESH:D001794': 'Blood Pressure',
  'MESH:D019936': 'Probiotics',
  'MESH:D010146': 'Pain',
  'MESH:D010496': 'Pericardium',
  'CHEBI:CHEBI:26195': 'polyphenol',
  'MESH:D011429': 'Propolis',
  'UP:P46639': 'KNAT1',
  'MESH:C017092': 'bromopyruvate',
  'MESH:D001996': 'Bronchopneumonia',
  'MESH:D010391': 'Pemphigoid, Bullous',
  'CHEBI:CHEBI:3219': 'bupropion',
  'MESH:D016957': 'Burkholderia pseudomallei',
  'CHEBI:CHEBI:85122': 'paraben',
  'CHEBI:CHEBI:88542': 'Butylparaben'},
 ['CHEBI:CHEBI:29865', 'MESH:D001794', 'MESH:D004164']]

In [26]:
excluded_longforms = ['bp']

In [27]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [28]:
additional_entities = {}

In [29]:
unambiguous_agent_texts = {}

In [30]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [31]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-03 04:02:29] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-03 04:06:26] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9870412939858071 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [32]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:29865': 301,
  'MESH:D001794': 6046,
  'ungrounded': 373,
  'MESH:D010391': 155,
  'CHEBI:CHEBI:18208': 5,
  'CHEBI:CHEBI:3044': 2,
  'CHEBI:CHEBI:35545': 3,
  'CHEBI:CHEBI:15373': 2,
  'CHEBI:CHEBI:17097': 11,
  'MESH:D001752': 10,
  'MESH:D010496': 11,
  'MESH:D004164': 97,
  'CHEBI:CHEBI:85122': 10,
  'MESH:D010146': 6,
  'MESH:D001714': 70,
  'UP:P46639': 29,
  'CHEBI:CHEBI:22901': 7,
  'MESH:D001416': 6,
  'MESH:C017092': 3,
  'CHEBI:CHEBI:3219': 4,
  'MESH:D020330': 4,
  'MESH:D018410': 3,
  'MESH:D016957': 1,
  'MESH:D010949': 3,
  'CHEBI:CHEBI:88542': 8,
  'MESH:D011429': 2,
  'MESH:D010477': 2,
  'CHEBI:CHEBI:33694': 3,
  'MESH:D019936': 2,
  'CHEBI:CHEBI:26195': 1,
  'MESH:D001996': 2},
 'f1': {'mean': 0.987041, 'std': 0.002767},
 'precision': {'mean': 0.981141, 'std': 0.004791},
 'recall': {'mean': 0.993328, 'std': 0.002168},
 'MESH:C017092': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std':

In [33]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [34]:
disamb.dump(model_name, results_path)

In [35]:
print(disamb.info())

Disambiguation model for BP

Produces the disambiguations:
	Back Pain	MESH:D001416
	Bell Palsy	MESH:D020330
	Bipolar Disorder	MESH:D001714
	Blast Crisis	MESH:D001752
	Blood Pressure*	MESH:D001794
	Bronchopneumonia	MESH:D001996
	Burkholderia pseudomallei	MESH:D016957
	Butylparaben	CHEBI:CHEBI:88542
	Diphosphonates*	MESH:D004164
	KNAT1	UP:P46639
	Pain	MESH:D010146
	Pemphigoid, Bullous	MESH:D010391
	Perfusion	MESH:D010477
	Pericardium	MESH:D010496
	Plasma	MESH:D010949
	Pneumonia, Bacterial	MESH:D018410
	Probiotics	MESH:D019936
	Propolis	MESH:D011429
	benzo[a]pyrene*	CHEBI:CHEBI:29865
	benzphetamine	CHEBI:CHEBI:3044
	benzylpenicillin	CHEBI:CHEBI:18208
	biomacromolecule	CHEBI:CHEBI:33694
	biopterin	CHEBI:CHEBI:15373
	biphenyl	CHEBI:CHEBI:17097
	bipyridine	CHEBI:CHEBI:35545
	bisphenol	CHEBI:CHEBI:22901
	bromopyruvate	MESH:C017092
	bupropion	CHEBI:CHEBI:3219
	paraben	CHEBI:CHEBI:85122
	polyphenol	CHEBI:CHEBI:26195

Class level metrics:
--------------------
Grounding                	Count	F1  

In [36]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1